In [7]:
def read_file(file_name_in):
    '''
    Read matrix from file
    
    Inputs:
        file_name_in : str
            The name of input file
            
    Outputs:
        marix : list (of list)
            Matrix which is read from file
    '''
    
    # YOUR CODE HERE
    with open(file_name_in, 'r') as f:
        readLines = f.readlines()
        lines = [line[:-1] for line in readLines if line[-1] == '\n']
        lines.append(readLines[-1])
        Matrix = []
        for i in lines:
            Matrix.append(list(i.split()))
        Matrix = [list(map(int, _)) for _ in Matrix]
        f.close()
    return Matrix

def write_file(file_name_out, determinant, inverse_matrix):
    '''
    Write determinant and inversed matrix into file
    
    Inputs:
        file_name_out : str
            The name of output file
            
        determinant : int or float
            The determinant of matrix read from input file
        
        inversed_matrix : list (of list)
            The inverse of matrix read from input file
    '''
    
    # YOUR CODE HERE
    with open(file_name_out, 'w') as file:
        file.write('Det: '+ str(determinant) + '\nInverse matrix:\n')
        if inverse_matrix is not None:
            [[file.write(str(i[j]) + ' ') if j != len(i) - 1 else file.write(str(i[j]) + '\n') for j in range(len(i))] for i in inverse_matrix]
        else: file.write('None')
        file.close()
        
# YOUR OTHER FUNCTIONS
import math

def is_zero(x):
    # kiểm tra số thực x có là số thực 0 (rất gần 0)
    return math.isclose(x, 0, abs_tol=1e-09)

def mul_scalar_vector(alpha, v):
    return [alpha*vi for vi in v]

def add_vector(v, w):
    return [vi + wi for vi, wi in zip(v, w)]

def row_switch(A, i, k):
    "di <-> dk"
    A[i], A[k] = A[k], A[i]
    
def row_mul(A, i, alpha):
    "di = anpha*di"
    A[i] = mul_scalar_vector(alpha, A[i])

def row_add(A, i, k, alpha):
    "di = di + anpha*dk"
    A[i] = add_vector(A[i], mul_scalar_vector(alpha, A[k]))
    
def Gauss_elimination(A):
    R = A.copy()
    m, n = len(R), len(R[0]) # kích thước ma trận
    
    sign = 1
    row = col = 0
    
    while row < m:
        # Bước 1
        while col < n and all(is_zero(R[i][col]) for i in range(row, m)):
            col += 1
        if col == n: # đã có dạng bậc thang
            break
        
        # Bước 2 (chọn dòng đầu tiên có số hạng khác 0)
        pivot_row = row + [not is_zero(R[i][col]) for i in range(row, m)].index(True)
        row_switch(R, row, pivot_row)
        if row != pivot_row:
            sign *= -1 
            
        # Bước 4
        for i in range(row + 1, m):
            multiplier = R[i][col]/R[row][col]
            row_add(R, i, row, -multiplier) # di = di - drow * multiplier
    
        # Bước 5
        row += 1
        
    return R, sign

def augmented_matrix(A, b):
    "Tạo ma trận bổ sung [A|b]"
    return [ai + bi for ai, bi in zip(A, b)]

def create_identity_matrix(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

def calc_determinant_row_operation(matrix):
    '''
    Calculate determinant of input matrix
    
    Inputs:
        marix : list (of list)
            Matrix which is read from file
            
    Outputs:
        determinant : int or float
            The determinant of input matrix
    '''
    
    # YOUR CODE HERE
    echelon_matrix, sign = Gauss_elimination(matrix)
    determinant = 1
    for i in range(len(echelon_matrix)):
        determinant *= echelon_matrix[i][i]
    
    return determinant * sign

def invert_matrix_row_operation(matrix):
    '''
    Invert a matrix
    
    Inputs:
        marix : list (of list)
            Matrix which is read from file
            
    Outputs:
        inverse_matrix : list (of list) or None
            The inverse of input matrix
            `None` when the input matrix is not invertible
    '''
    
    # YOUR CODE HERE
    # Check the matrix is inversable or not
    if calc_determinant_row_operation(matrix) == 0:
        return None
    aug_matrix = augmented_matrix(matrix, create_identity_matrix(len(matrix)))
    # Change the agumented matrix to the echelon form
    echelon_aug, sign = Gauss_elimination(aug_matrix)
    # Change the number in the main diagonal to 1
    for i in range(len(echelon_aug)):
        temp = echelon_aug[i][i]
        for j in range(len(echelon_aug[0])):
            echelon_aug[i][j] = echelon_aug[i][j] / temp
    # Change all elements above the main diagonal of the identity part to 0
    for i in range(len(echelon_aug)):
        for j in range(len(echelon_aug)):
            if i != j:
                row_add(echelon_aug, j, i, -(echelon_aug[j][i] / echelon_aug[i][i]))
    # Delete the identity matrix part
    inverse = [row[len(matrix):] for row in echelon_aug]
    
    return inverse

def main():
    matrix = read_file(file_name_in='input.txt')
    
    det = calc_determinant_row_operation(matrix)
    inv_mat = invert_matrix_row_operation(matrix)
    
    write_file(file_name_out='19127216_output.txt', determinant=det, inverse_matrix=inv_mat)
    
main()